# <a name="C0">SOMMAIRE</a>


####  <a href="#C1">Set up environnement</a>
#### <a href="#C2">Sélection des variables qui nous intéressent</a>
#### <a href="#C3">Nettoyage</a>
- <a href="#C31">On enlève les produits avec pas assez d'info</a>
- <a href="#C32">On traite les duplicates</a>
- <a href="#C33">On traite les valeurs aberrantes</a>
- <a href="#C34">On traite les valeurs manquantes</a>
 
 
 
### Ce qu'il faut faire :

#### 1) Nettoyage :
    - lire definitions des variables (ex nutriscore)
    - idée d'application (inspi kaggle ? par catégorie par pays mots clefs pour regrouper ?)
    - filtrer jeu données , seule var indispensable à mon application
    - traiter valeurs aberrantes // contexte projet 
    - valeurs manquantes :
         - remplacer données manquantes (0 moy mediane)
         - estimer avec algo KNN ou
         - fonction IterativeImputer (only via corrélation)
#### 2) Analyse exploratoire
    - analyses univariées ( describe.. graph numérique ou catégorielle)
    - bivariées ( graphiques croisant var numérique et une catégorielle OU croisant 2 numériques + matrice de corrélation)
    => matplotlib et seaborn, ex illustrer nb articles vendus ds chaque pays, répartition nutriscore ds chacun d'entre eux
    - multivariées 
    => descriptive avec ACP Analyse en Composantes Principales
    => explicative avec l'ANOVA
#### 3) Présentation de l'idée d'application ( c'est tout )
    - un graph présenté = une analyse = vérifier une hypothèse ou déclencher une action sur les données
    

# <a name="C1">Set up environnement</a>
<a href="#C0">SOMMAIRE</a>

In [1]:
#Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from datetime import datetime as dt
import datetime
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer, KNNImputer
from numpy import isnan
%matplotlib inline

In [2]:
#Import des données
OFFproducts = pd.read_csv('C:/Users/Me/Documents/1- DATA SCIENCE/fr.openfoodfacts.org.products.csv',sep='\t',error_bad_lines=False)

C:\Users\Me\AppData\Local\Temp\ipykernel_15628\2303218264.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  OFFproducts = pd.read_csv('C:/Users/Me/Documents/1- DATA SCIENCE/fr.openfoodfacts.org.products.csv',sep='\t',error_bad_lines=False)
C:\Users\Me\AppData\Local\Temp\ipykernel_15628\2303218264.py:2: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  OFFproducts = pd.read_csv('C:/Users/Me/Documents/1- DATA SCIENCE/fr.openfoodfacts.org.products.csv',sep='\t',error_bad_lines=False)


In [3]:
#Réglages d'affichage
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 2000)

# Rappel du thème utilisé
sns.set_style('darkgrid')
sns.set_palette('Paired')

# <a name="C2">Sélection des variables qui nous intéressent</a>
<a href="#C0">SOMMAIRE</a>


Première idée d'application : suggérer l'équivalent le + ecofriendly ou avec le meilleur nutriscore ou bien vegan ?


A affiner selon les données dont nous disposons.


In [4]:
OFFproducts.head(1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000000003087/farine-de-ble-noir-ferme-t-y-r-nao,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-completed, en:ingredients-to-be-completed, en:expiration-date-to-be-completed, en:characteristics-to-be-completed, en:categories-to-be-completed, en:brands-completed, en:packaging-to-be-completed, en:quantity-completed, en:product-name-completed, en:photos-to-be-validated, en:photos-uploaded","en:to-be-completed,en:nutrition-facts-to-be-completed,en:ingredients-to-be-completed,en:expiration-date-to-be-completed,en:characteristics-to-be-completed,en:categories-to-be-completed,en:brands-completed,en:packaging-to-be-completed,en:quantity-completed,en:product-name-completed,en:photos-to-be-validated,en:photos-uploaded","A compléter,Informations nutritionnelles à compléter,Ingrédients à compléter,Date limite à compléter,Caractéristiques à compléter,Catégories à compléter,Marques complétées,Emballage à compléter,Quantité complétée,Nom du produit complete,Photos à valider,Photos envoyées",NaN,NaN,NaN,N

In [ ]:
# on garde seulement les variables remplies à plus de 60%, je prends pour base le fait que je veux le nutrition score qui est à 68
OFFproducts.info()
remplOFFp = round(100-(OFFproducts.isnull().sum()/len(OFFproducts)*100),2)
print(remplOFFp)
listgarder = remplOFFp[remplOFFp>=60].index
#je duplique mes données et je ne garde que les variables remplies à plus de 60%
data = OFFproducts[listgarder]
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB


In [ ]:
sns.set_style('darkgrid')
sns.set_palette('Set2')
remplOFFp.sort_values(inplace=True)
plt.figure(figsize=(6,27))
plt.title('Taux de remplissage des variables')
remplOFFp.plot(kind='barh')
plt.show()

### Est-ce que les données sont + ou - à jour ?

In [ ]:
# je m'assure du format
data = data.dropna(subset=['code'])
data['last_modified_datetime']= pd.to_datetime(data['last_modified_datetime'],errors = 'ignore')
data['last_modified_year']=data['last_modified_datetime'].dt.year
data['last_modified_year'].value_counts().plot(kind='bar')

### Quels indicateurs va-t-on garder ?
Je choisis de ne garder qu'une quinzaine d'indicateurs

**Les indicateurs obligatoires : 

1) 'code' : car c'est la clé du produit

2) 'url' : indispendable pour trouver les infos sur le produit

3) 'product_name' : indispensable 

4) 'brands'

**=> Sur la partie emballage pas assez de données donc j'écarte cette idée

C'est une application FR donc je vais devoir quand même inscrire si oui ou non les **produits sont disponibles en France**, il faudrait qu'on puisse scanner les produits de l'étranger mais qu'on ne puisse pas recevoir en résultat un produit indisponible en France

5) 6) 7) 'countries', 'countries_tags', 'countries_fr'  (on en enlevera à la fin)


**Sur la partie ingrédients au cas ou je veux donner des alternatives végétales ?**

8) ingredients_text

9) 10) 'additives_n', 'additives'  : pour la présence d'additifs (pas bon pour la santé)

**Composition calories** etc : je vais me référer à ce que l'on trouve en mention obligatoire sur les paquets :

11) 12) 13) 14) 15) 16) 17) 18) 19) 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g'

**La base de ce que je veux produire pour l'application avec la meilleure alternative**

 20) 21) 'nutrition-score-fr_100g',
        'nutrition_grade_fr'
       
       
**Et j'ajoute** 'pnns_groups_1', 'pnns_groups_2',  'main_category_fr' même si elles sont très peur remplies car cela peut être utile pour l'analyse....

In [ ]:
#liste des indicateurs que l'on garde avant traitement
listgarder

In [ ]:
# création de la liste des variables que je vais vraiment garder après réflexion :
list_var = ['code','url' ,'product_name' ,'brands', 
                  'countries', 'countries_tags', 'countries_fr',
                  'ingredients_text','additives_n', 'additives',
                   'energy_100g', 'fat_100g','saturated-fat_100g', 'carbohydrates_100g', 
                  'sugars_100g', 'fiber_100g','proteins_100g', 'salt_100g', 'sodium_100g',
                  'nutrition-score-fr_100g','nutrition_grade_fr',
                  'pnns_groups_1', 'pnns_groups_2',  'main_category_fr']
df = OFFproducts[list_var]
                  

In [ ]:
df.head(1)

# <a name="C3">Nettoyage</a>

<a href="#C0">SOMMAIRE</a>

## <a name="C31">On enlève les produits avec pas assez d'info</a>

In [ ]:
#je remplace dans tout mon df les valeurs vides par NaN
df.replace('',np.nan,inplace=True)

In [ ]:
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

In [ ]:
#affichage des valeurs nulles pour la var code
display(df[df['code'].isnull()])
#on dirait que ce sont des catégories de types de yahourt. Pas utile pour nous nous droppons donc les NaN pour le Code
df = df.dropna(subset=['code'])


In [ ]:
#affichage des valeurs nulles pour la var product_name
display(df[df['product_name'].isnull()])
dfpnn = df[df['product_name'].isnull()]
#Ce sont des informations vraiemnt incomplètes à première vue... Affichons le taux de remplissage des autres valeurs
# au sein de ce dataframe 
print('taux de remplissage quand le produit_name est vide')
print(100-(dfpnn.isnull().sum()/len(dfpnn)*100))
print('taux de remplissage quand le produit_name est vide par rapport aux taux de remplissage du dataset')
print((100-(dfpnn.isnull().sum()/len(dfpnn)*100))/(100-(df.isnull().sum()/len(df)*100))*100)
print('nombre de valeurs manquantes :', len(dfpnn))
# comme c'est très peu rempli et qu'on a peu d'info même pas le nom du produit je décide de supprimer ces lignes
df = df.dropna(subset=['product_name'])

In [ ]:
len(df[df['brands'].isnull()])
# 13k lignes ça paraît compliqué de les remplir à la main mais on remarque des lignes test ...
display(df[df['product_name'].str.contains('Test')])
# on créer la liste des noms des produits test
test=['Test','Test nf','Pierretest TestTestTestTestTestTTestTestTestTestTestTestTesttTestTestTestTestTestTestTestTestTestTestTestTestTestTesttTestTestTestTestTestTestTestTestTestTestTestTestTestTest',
    'Test product','Test_edition','Test NF App']
for i in test :
    df.drop(df[df['product_name']==i].index, inplace=True)


In [ ]:
print(len(df))
print('_')
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

In [ ]:
list(df)

In [ ]:
#j'enlève les lignes ou à partir des ingrédients il n'y a auncune donnée
a=['ingredients_text',
 'additives_n',
 'additives',
 'energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'nutrition-score-fr_100g',
 'pnns_groups_1',
 'pnns_groups_2',
 'main_category_fr']
print(a)
print('nb de lignes sans données ci-dessus:',len(df)-len(df.dropna(subset=a,how='all')))
print('cela représente en % :',(len(df)-len(df.dropna(subset=a,how='all')))/len(df)*100)
df=df.dropna(subset=a,how='all')


## <a name="C32">On traite les duplicates</a>
<a href="#C0">SOMMAIRE</a>

In [ ]:
#trouver les doublons sur toutes les var
df[df.duplicated(keep=False)] #il n'y en a pas
# sur un subset product_name
duplicname = df[df.duplicated(subset='product_name',keep=False)]
print('nombres de duplicates pour product_name :',len(duplicname))
print('nombres de valeurs uniques pour les duplicates pour product_name :',len(duplicname['product_name'].unique()))
# sur un subset product_name x brands
duplicnamebrand = df[df.duplicated(subset=['product_name','brands'],keep=False)]
print('nombres de duplicates pour product_name x brands:',len(duplicnamebrand))
# sur un subset code
dupliccode = df[df.duplicated(subset='code',keep=False)]
print('nombres de duplicates pour code :',len(dupliccode))
# sur un subset url
duplicurl = df[df.duplicated(subset='url',keep=False)]
print('nombres de duplicates pour url :',len(duplicurl))

In [ ]:
#créer une colonne qui compte les valeurs nulles
df['countisnull']=pd.isnull(df).sum(1)
#je drop les duplicates sur la colonne code en gardant la ligne la plus remplie à chaque fois puis je supprime la colonne countisnull
df= df.sort_values(['countisnull']).drop_duplicates(subset=['code'],keep='first').drop('countisnull',1)

In [ ]:
len(df)

In [ ]:
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

## <a name="C33">On traite les valeurs aberrantes</a>
<a href="#C0">SOMMAIRE</a>

In [ ]:
#valeurs aberrantes ? Détection d'outliers ? pour les variables quantitatives seulement
df.describe()

Valeurs en grammes : 'fat_100g','saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
            'salt_100g', 'sodium_100g', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g'

autres valeurs : 'energy_100g' c'est en calories donc il faut un traitement différent.
Après des recherches en ligne l'aliment le plus calorique c'est l'huile et cela fait 3765.6 kj, ainsi nous allons partir du principe qu'il est impossible d'avoir des valeurs supérieures à 4000.

In [ ]:
#'energy_100g'
#on remplace les valeurs > 4000 par NaN
print(len(df[df['energy_100g']>4000.0]))
display(df[df['energy_100g']>4000.0])
df.loc[(df['energy_100g']>4000),'energy_100g']=np.nan

In [ ]:
# boucle pour remplacer les valeurs aberrantes 

for var in ['fat_100g','saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
            'salt_100g', 'sodium_100g', 'nutrition-score-fr_100g'] :
    df.loc[abs(df[var])>100,var]=np.nan # on remplace par NaN les valeurs au delà de 100 et de -100
    df.loc[(df[var]<0)&(abs(df[var])<100),var]=abs(df[(df[var]<0)&(abs(df[var])<100)][var]) #on remplace par la val absolue
    #les valeurs négatives qui sont comprises entre 0 et -100

In [ ]:
df.reset_index(drop=True,inplace=True)

## <a name="C34">On traite les valeurs manquantes</a>
<a href="#C0">SOMMAIRE</a>



remplacer par médiane / moy de la colonne ou du pnn group
iterate imputer par ex pour nutrition score
knn imputer pour energy 100g

#### On remplace les valeurs vides par la médiane du pnn group pour les valeurs en 100g

In [ ]:
print('pnns_group_1 :',df['pnns_groups_1'].unique())
print('pnns_group_2 :',df['pnns_groups_2'].unique())
# on remplace unknow par NaN
#df.loc[df['pnns_groups_1']=='unknown','pnns_group_1']=np.nan
#df.loc[df['pnns_groups_2']=='unknown','pnns_group_2']=np.nan
df.replace(to_replace={
             'pnns_groups_1':{'unknown':np.nan}, 
             'pnns_groups_2':{'unknown':np.nan},},inplace=True)

In [ ]:
#pré-traitement pour 'pnns_groups_1'
tofind = ['salty-snacks','cereals-and-potatoes','fruits-and-vegetables','sugary-snacks']
toreplace = ['Salty snacks','Cereals and potatoes','Fruits and vegetables','Sugary snacks']
for x,y in zip(tofind,toreplace) :
    df["pnns_groups_1"]=df["pnns_groups_1"].replace(x,y)


In [ ]:
compoalim = ['fat_100g','saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
            'salt_100g', 'sodium_100g']
print('% des valeurs NaN avec le pnn rempli sur la part totale des valeurs NaN pour chaque variable que je souhaite remplir')
for var in compoalim :
    print(var,round(len(df[(df[var].isnull())&(~(df['pnns_groups_1'].isnull()))])/len(df[(df[var].isnull())])*100,2),'%')
    
print("\n nombre d'occurence de chaque catégorie du pnns_groups_2")
for categ in df['pnns_groups_2'].unique() :
    dfcateg=df[df['pnns_groups_2']==categ]
    print(categ,len(dfcateg))


In [ ]:
print(len(df))
print('_')
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

In [ ]:
# pour toutes les valeurs manquantes dans la composition alimentaire je choisis de remplacer dans un premier temps par 
#la médiane du groupe quand elle est disponible, autrement on met la moyenne
for var in compoalim :
    for categ in df['pnns_groups_2'].unique() :
        df.loc[(df[var].isnull())&(df['pnns_groups_2']==categ),var]=df[df['pnns_groups_2']==categ][var].median()
    df.loc[df[var].isnull(),var]=df[var].mean()

In [ ]:
print(len(df))
print('_')
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

#### iterate imputer pour energy_100g

In [ ]:
df.reset_index(drop=True)
dftest=df[['code','energy_100g','fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','fiber_100g',
          'proteins_100g','salt_100g','sodium_100g','code']]

In [ ]:
dftest

In [ ]:
# split into input and output elements
dftestdata = dftest.values
ix = [i for i in range(dftestdata.shape[1]) if i != 10]
X, y = dftestdata[:, ix], dftestdata[:, 10]
# print total missing
print('Missing: %d' % sum(pd.isnull(X).flatten()))
# define imputer
imputer = IterativeImputer(min_value=0,max_value=4000)
# fit on the dataset
imputer.fit(X)
# transform the dataset
Xtrans = imputer.transform(X)
# print total missing
print('Missing: %d' % sum(pd.isnull(Xtrans).flatten()))

In [ ]:
dftestii = pd.DataFrame (Xtrans,columns=['code','energy_100g','fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','fiber_100g',
          'proteins_100g','salt_100g','sodium_100g'])

In [ ]:
df.describe()

In [ ]:
dftestii.describe()

In [ ]:
sns.histplot(data=df['energy_100g'],color='gray')
sns.histplot(data=dftestii['energy_100g'],color='pink')
plt.legend(['distribution originale', 'distribution après imputation'])
plt.show()

In [ ]:
df['energy_100g']=dftestii['energy_100g']

In [ ]:
print(len(df))
print('_')
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

#### knn imputer pour nutrition-score-fr_100g

In [ ]:
df

In [ ]:
print(df['nutrition-score-fr_100g'].unique())

In [ ]:
df.reset_index(drop=True)
dftest2=df[['energy_100g','fat_100g','sugars_100g','proteins_100g','salt_100g','nutrition-score-fr_100g']]
imputer=KNNImputer(n_neighbors=1)
# print total missing
X=dftest2
print('Missing: %d' % sum(pd.isnull(X).values.flatten()))
Xtrans=imputer.fit_transform(X)
print('Missing: %d' % sum(pd.isnull(Xtrans).flatten()))

In [ ]:
dftestknn = pd.DataFrame (Xtrans,columns=['energy_100g','fat_100g','sugars_100g','proteins_100g','salt_100g','nutrition-score-fr_100g'])
dftestknn.describe()

In [ ]:
sns.histplot(data=df['nutrition-score-fr_100g'],color='gray')
sns.histplot(data=dftestknn['nutrition-score-fr_100g'],color='pink')
plt.legend(['distribution originale', 'distribution après imputation'])
plt.show()


In [ ]:
df['nutrition-score-fr_100g']=dftestknn['nutrition-score-fr_100g']
print(len(df))
print('_')
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

### application au nutrigrade_fr

In [ ]:
df['nutrition_grade_fr'].unique()

In [ ]:
#mapping the nutrition_grade_fr values
dftest3=df[['nutrition-score-fr_100g','nutrition_grade_fr']]
dftest3['nutrition_grade_map']=dftest3.nutrition_grade_fr.map({'a':1,'b':2,'c':3,'d':4,'e':5})
dftest3.drop('nutrition_grade_fr',axis=1,inplace=True)
dftest3

In [ ]:
imputer=KNNImputer(n_neighbors=1)
X=dftest3
print('Missing: %d' % sum(pd.isnull(X).values.flatten()))
Xtrans=imputer.fit_transform(X)
print('Missing: %d' % sum(pd.isnull(Xtrans).flatten()))

In [ ]:
dftestknn2 = pd.DataFrame (Xtrans,columns=['nutrition-score-fr_100g','nutrition_grade_map'])

In [ ]:
dftestknn2

In [ ]:
sns.histplot(data=dftest3['nutrition_grade_map'],color='gray')
sns.histplot(data=dftestknn2['nutrition_grade_map'],color='pink')
plt.legend(['distribution originale', 'distribution après imputation'])
plt.show()

In [ ]:
#mapping the nutrition_grade_fr values, dans l'autre sens
dftestknn2['nutrition_grade_fr']=dftestknn2.nutrition_grade_map.map({1:'a',2:'b',3:'c',4:'d',5:'e'})
df['nutrition_grade_fr']=dftestknn2['nutrition_grade_fr']

In [ ]:
print(len(df))
print('_')
print('Taux de remplissage des variables sélectionnées')
print(100-(df.isnull().sum()/len(df)*100))

## SAUVEGARDE DU FICHIER POUR L'ANALYSE EXPLORATOIRE

In [ ]:
df.to_csv('datapostnettoyagefood.csv', index=False)